In [107]:
import numpy as np
import csv
import math

train_file = 'train.csv'
test_file  = 'test.csv'
soln_file  = 'user_median.csv'

#parameters
train_size = 1
K = 5
learn_rate = 0.000002
learn_rate2 =0.000002
iterations = 7
penalty = .02
epochs = 10
init = 40


#load CSV
train_data = {}
test_data = {}
ndata = 0
with open(train_file, 'r') as train_fh:
    ndata = sum(1 for row in train_fh) - 1
    
splitting = np.random.binomial(1, train_size, size=ndata)


In [108]:
with open(train_file, 'r') as train_fh:
    train_csv = csv.reader(train_fh, delimiter=',', quotechar='"')
    next(train_csv, None)
    
    for i, row, in enumerate(train_csv):
        user   = row[0]
        artist = row[1]
        plays  = row[2]

        if splitting[i]: 
            if not user in train_data:
                train_data[user] = {}
            train_data[user][artist] = int(plays)
            
        else:
            if not user in test_data:
                test_data[user] = {}

            test_data[user][artist] = int(plays)
              
# Compute the global median and per-user median.
plays_array  = []
user_medians = {}
for user, user_data in train_data.iteritems():
    user_plays = []
    for artist, plays in user_data.iteritems():
        plays_array.append(plays)
        user_plays.append(plays)

    user_medians[user] = np.median(np.array(user_plays))
global_median = np.median(np.array(plays_array))

In [109]:
user_mat = []
artist_mat = []

user_bias = {}
artist_bias = {}



for k in range(K):
    print "\r", k,
    user_mat.append({})
    artist_mat.append({})
    
    for user, user_data in train_data.iteritems():
        if not user in user_mat[k]:
            user_mat[k][user] = np.random.rand()
        if not user in user_bias:
            user_bias[user] = init*np.random.rand()
        for artist, plays in user_data.iteritems():
            if not artist in artist_bias:
                artist_bias[artist] = init*np.random.rand()
            if not artist in artist_mat[k]:
                artist_mat[k][artist] = np.random.rand()

5


In [110]:
def predict(user, artist):
    return sum([user_mat[k][user] * artist_mat[k][artist] for k in range(K)]) + user_bias[user] + artist_bias[artist]

def modify(user, artist, prediction):
    newpred = (1 - weight_user_median)*prediction + weight_user_median*(user_medians[user])
    if newpred < 0:
        return 0
    else:
        return newpred

def train_biases(data):
    for e in range(epochs):
        for i, (user, user_data) in enumerate(train_data.iteritems()):
                if i % 10000 == 0:
                    print "\r", e,epochs,i,

                for artist, plays in user_data.iteritems():  
                    artist_bias[artist] += learn_rate2*(plays - predict(user,artist))
                    user_bias[user] += learn_rate2*(plays - predict(user,artist))
                    
def train(data):
    for i in range(iterations):
        for k in range(K):
            user_feature = user_mat[k]
            artist_feature = artist_mat[k]

            for j, (user, user_data) in enumerate(train_data.iteritems()):
                if j % 10000 == 0:
                    print "\r", i,j,k,K,

                for artist, plays in user_data.iteritems(): 
                    user_feature[user] += learn_rate * (artist_feature[artist] * (plays - predict(user, artist)) - penalty*user_feature[user])
                    artist_feature[artist] += learn_rate * (user_feature[user] * (plays - predict(user, artist)) - penalty*artist_feature[artist])

          
                
           


In [111]:
%%time

train_biases(train_data)

9 10 230000CPU times: user 4min 17s, sys: 330 ms, total: 4min 17s
Wall time: 4min 19s



In [ ]:
%%time
train_size = 1
K = 5
learn_rate = 0.000002
learn_rate2 =0.000002
iterations = 7
penalty = .02
epochs = 10
init = 40
train(train_data)

0 80000 0 5 

In [ ]:
def calc_error(data):
    errors = []

    for j, (user, user_data) in enumerate(data.iteritems()):
        if j % 10000 == 0:
            print "\r", j,
        for artist, plays in user_data.iteritems():
            pred = predict(user,artist)
            
            newpred = modify(user,artist,pred)
            error = abs(plays - newpred)
            errors.append(error)
    return (sum(np.array(errors))/len(errors))

weight_user_median = .95
#print calc_error(test_data)


In [ ]:

with open(test_file, 'r') as test_fh:
    with open('predictions_5.csv', 'w') as test_output:
        test_csv = csv.reader(test_fh, delimiter=',', quotechar='"')
        next(test_csv, None)
        writer = csv.writer(test_output)
        writer.writerow(('Id', 'plays'))

        for i, row in enumerate(test_csv):
            if i % 10000 == 0:
                print '\r', i,
            pred_id = row[0]
            user = row[1]
            artist = row[2]
            pred = predict(user,artist)
            newpred = modify(user,artist,pred)
            writer.writerow((pred_id, newpred))